<h1>sign laungauge interpertation</h1>



<h3>Import libraryes</h3>

In [15]:
import tensorflow as tf
import cv2
import matplotlib as plt
import numpy as np
import time
import mediapipe as mp

<h3>Recording frames & getting Landmarks</h3>


In [17]:
mp_hol=mp.solutions.holistic
mp_draw=mp.solutions.drawing_utils

In [43]:
cam=cv2.VideoCapture(0)

with mp_hol.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hol:
    while cam.isOpened():
        ret,frame=cam.read()
        frame=cv2.flip(frame,1)
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        landmarks=hol.process(image)
           
        #drawing on image
        mp_draw.draw_landmarks(frame,landmarks.face_landmarks,mp_hol.FACEMESH_CONTOURS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))
        mp_draw.draw_landmarks(frame,landmarks.left_hand_landmarks,mp_hol.HAND_CONNECTIONS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))
        mp_draw.draw_landmarks(frame,landmarks.right_hand_landmarks,mp_hol.HAND_CONNECTIONS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))
        mp_draw.draw_landmarks(frame,landmarks.pose_landmarks,mp_hol.POSE_CONNECTIONS,
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                              mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1))
        if not ret:
            break
        cv2.imshow("capture",frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cam.release()
    cv2.destroyAllWindows()


In [31]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
